In [ ]:
!

In [1]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [1]:
import pandas as pd
import os
import gensim
from tqdm import tqdm
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import nltk
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import utils
from sklearn.metrics import accuracy_score, f1_score
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import LabelEncoder
import numpy as np
import re
from sklearn.model_selection import GridSearchCV

In [2]:
labeled_features = pd.read_excel("bag_of_words_v1.xlsx")
labeled_features_v1 = pd.read_excel("bag_of_words_v2.xlsx")
test_label_features = pd.read_excel("test_bag_of_words.xlsx")
unlabeled_data = pd.read_csv("Training Dataset/unlabeled_data.csv")
test_file = pd.read_excel("kaggle_test_bag_of_words.xlsx")

In [3]:
labeled_features_v1.head()

,text,label,sad,smile,laugh,very_happy,crying,tongueOut,wink,kiss,surprise,disgust,exclamation,capital_Ratio
0,the_new rule is if you_are waiting_for table w...,4,0,0,0,0,0,0,0,1,0,0,0,0.029010
1,flirted with giving this two_star but_that is ...,3,0,0,0,0,0,0,0,0,0,0,0,0.019856
2,wa staying at planet hollywood across_the stre...,5,0,1,0,0,0,0,0,0,0,0,0,0.018550
3,food is good_but price_are super expensive buc...,2,0,0,0,0,0,0,0,0,0,0,0,0.021307
4,worse company to deal with they do horrible wo...,1,0,0,0,0,0,0,0,0,0,0,0,0.011024


In [23]:
unlabeled_data.head(3)

,text
0,Had a good experience when my wife and I sat a...
1,On my first to Montreal with my gf we came her...
2,One of our favorite places to go when it's col...


In [ ]:
test_label_features.head()

In [4]:
 labeled_features_v1[labeled_features_v1['text'].isnull()]

,text,label,sad,smile,laugh,very_happy,crying,tongueOut,wink,kiss,surprise,disgust,exclamation,capital_Ratio
1491,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0.0
29785,NaN,2,0,0,0,0,0,0,1,0,0,0,0,0.0


In [24]:
unlabeled_data[unlabeled_data['text'].isnull()]

,text


In [7]:
test_label_features[test_label_features['text'].isnull()]

,text,label,sad,smile,laugh,very_happy,crying,tongueOut,wink,kiss,surprise,disgust,exclamation,capital_Ratio


In [29]:
test_file[test_file['text'].isnull()]

,text,sad,smile,laugh,very_happy,crying,tongueOut,wink,kiss,surprise,disgust,exclamation,capital_Ratio
39384,NaN,0,0,0,0,0,0,0,0,0,0,0,0.00289


In [5]:
# Drop NaNs

# labeled_features = labeled_features.dropna()
labeled_features_v1 = labeled_features_v1.dropna()

In [25]:
# X_train = labeled_features['text']
# Y_train = labeled_features['label']

X_train = labeled_features_v1['text']
Y_train = labeled_features_v1['label']

X1_test = unlabeled_data['text']

In [8]:
X_test = test_label_features['text']
Y_test = test_label_features['label']

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_vectoriser = TfidfVectorizer(use_idf = True)

x_train = tf_vectoriser.fit_transform(X_train)
x_test = tf_vectoriser.transform(X_test)
x1_test = tf_vectoriser.transform(X1_test)

print("Vocab size: {}".format(len(tf_vectoriser.vocabulary_)))
print("x_train: {}".format(repr(x_train)))
print("x_test: {}".format(repr(x_test)))

feature_names = tf_vectoriser.get_feature_names()
print("Number of features: {}".format(len(feature_names)))

Vocab size: 49117
x_train: <39998x49117 sparse matrix of type '<class 'numpy.float64'>'
	with 3071913 stored elements in Compressed Sparse Row format>
x_test: <10000x49117 sparse matrix of type '<class 'numpy.float64'>'
	with 768433 stored elements in Compressed Sparse Row format>
Number of features: 49117


In [ ]:
# column_name = x_train.indices
# print("Column name: ", column_name)

In [27]:
x_train.get_shape()

(39998, 49117)

In [28]:
x_test.get_shape()

(10000, 49117)

In [ ]:
# first_vector_tfidfvectorizer=x_train[10]
# df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tf_vectoriser.get_feature_names(), columns=["tfidf"])
# df.sort_values(by=["tfidf"],ascending=False)

In [12]:
# Adding emoticons for training

sad = np.array(labeled_features_v1['sad'])
print("Sad:",sad.shape)
smile = np.array(labeled_features_v1['smile'])
print("Smile:",smile.shape)
laugh = np.array(labeled_features_v1['laugh'])
print("Laugh",laugh.shape)
very_happy = np.array(labeled_features_v1['very_happy'])
print("Very happy:",very_happy.shape)
crying = np.array(labeled_features_v1['crying'])
print("crying:",crying.shape)
tongueOut = np.array(labeled_features_v1['tongueOut'])
print("tongueOut:",tongueOut.shape)
wink = np.array(labeled_features_v1['wink'])
print("wink:",wink.shape)
kiss = np.array(labeled_features_v1['kiss'])
print("kiss:",kiss.shape)
surprise = np.array(labeled_features_v1['surprise'])
print("surprise:",surprise.shape)
disgust = np.array(labeled_features_v1['disgust'])
print("disgust:",disgust.shape)
exclamation = np.array(labeled_features_v1['exclamation'])
print("exclamation:",exclamation.shape)
capital_Ratio = np.array(labeled_features_v1['capital_Ratio'])
print("capital_Ratio:",capital_Ratio.shape)

Sad: (39998,)
Smile: (39998,)
Laugh (39998,)
Very happy: (39998,)
crying: (39998,)
tongueOut: (39998,)
wink: (39998,)
kiss: (39998,)
surprise: (39998,)
disgust: (39998,)
exclamation: (39998,)
capital_Ratio: (39998,)


In [13]:
# Adding emoticons for TEST

sad1 = np.array(test_label_features['sad'])
print("Sad:",sad1.shape)
smile1 = np.array(test_label_features['smile'])
print("Smile:",smile1.shape)
laugh1 = np.array(test_label_features['laugh'])
print("Laugh",laugh1.shape)
very_happy1 = np.array(test_label_features['very_happy'])
print("Very happy:",very_happy1.shape)
crying1 = np.array(test_label_features['crying'])
print("crying:",crying1.shape)
tongueOut1 = np.array(test_label_features['tongueOut'])
print("tongueOut:",tongueOut1.shape)
wink1 = np.array(test_label_features['wink'])
print("wink:",wink1.shape)
kiss1 = np.array(test_label_features['kiss'])
print("kiss:",kiss1.shape)
surprise1 = np.array(test_label_features['surprise'])
print("surprise:",surprise1.shape)
disgust1 = np.array(test_label_features['disgust'])
print("disgust:",disgust1.shape)
exclamation1 = np.array(test_label_features['exclamation'])
print("exclamation:",exclamation1.shape)
capital_Ratio1 = np.array(test_label_features['capital_Ratio'])
print("capital_Ratio:",capital_Ratio1.shape)

Sad: (10000,)
Smile: (10000,)
Laugh (10000,)
Very happy: (10000,)
crying: (10000,)
tongueOut: (10000,)
wink: (10000,)
kiss: (10000,)
surprise: (10000,)
disgust: (10000,)
exclamation: (10000,)
capital_Ratio: (10000,)


In [14]:
from scipy.sparse import hstack

X_train_dtm = hstack((x_train,sad[:,None]))
X_train_dtm = hstack((X_train_dtm,smile[:,None]))
X_train_dtm = hstack((X_train_dtm,laugh[:,None]))
X_train_dtm = hstack((X_train_dtm,very_happy[:,None]))
X_train_dtm = hstack((X_train_dtm,crying[:,None]))
X_train_dtm = hstack((X_train_dtm,tongueOut[:,None]))
X_train_dtm = hstack((X_train_dtm,wink[:,None]))
X_train_dtm = hstack((X_train_dtm,kiss[:,None]))
X_train_dtm = hstack((X_train_dtm,surprise[:,None]))
X_train_dtm = hstack((X_train_dtm,disgust[:,None]))
X_train_dtm = hstack((X_train_dtm,exclamation[:,None]))
X_train_dtm = hstack((X_train_dtm,capital_Ratio[:,None]))

In [15]:
from scipy.sparse import hstack

X_test_dtm = hstack((x_test,sad1[:,None]))
X_test_dtm = hstack((X_test_dtm,smile1[:,None]))
X_test_dtm = hstack((X_test_dtm,laugh1[:,None]))
X_test_dtm = hstack((X_test_dtm,very_happy1[:,None]))
X_test_dtm = hstack((X_test_dtm,crying1[:,None]))
X_test_dtm = hstack((X_test_dtm,tongueOut1[:,None]))
X_test_dtm = hstack((X_test_dtm,wink1[:,None]))
X_test_dtm = hstack((X_test_dtm,kiss1[:,None]))
X_test_dtm = hstack((X_test_dtm,surprise1[:,None]))
X_test_dtm = hstack((X_test_dtm,disgust1[:,None]))
X_test_dtm = hstack((X_test_dtm,exclamation1[:,None]))
X_test_dtm = hstack((X_test_dtm,capital_Ratio1[:,None]))

In [16]:
X_train_dtm.get_shape()

(39998, 49129)

In [17]:
X_test_dtm.get_shape()

(10000, 49129)

### Logistic regression

In [18]:
clf = LogisticRegression()
clf.fit(X_train_dtm, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [19]:
y_pred = clf.predict(X_test_dtm)

In [20]:
from sklearn.metrics import accuracy_score

print("Accuracy : ", accuracy_score(Y_test, y_pred))

Accuracy :  0.6025


In [22]:
type(Y_train)

pandas.core.series.Series

### Extracting 50k records from unlabeled data and predicting the label

In [1]:
# extracting first 50k records from unlabeled data 

# Converting unlabeled data to dataframe
X1_test_df = X1_test.to_frame()

# Resetting the index of the dataframe
X1_test_df.reset_index(inplace = True)

# Dropping first column
X1_test_df.drop('index', axis = 1, inplace = True)

# new_data = X1_test_lr.sample(frac=0.1, replace = False)
new_data = X1_test_df.iloc[0:50000,:]

# converting the extracted records to series
new_data = new_data.T.squeeze()

NameError: name 'X1_test' is not defined